# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl')

IBM Watson

In [4]:
!pip install watson-machine-learning-client --upgrade

  Using cached https://files.pythonhosted.org/packages/12/67/66db412f00d19bfdc5725078bff373787513bfb14320f2804b9db3abb53a/watson_machine_learning_client-1.0.378-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/b1/02eebed49c754b01b17de7705caa8c4ceecfb4f926cdafc220c863584360/lomond-0.3.3-py2.py3-none-any.whl
Processing c:\users\usuario\appdata\local\pip\cache\wheels\4b\d1\47\7bfbd0c47216fbf180dd8e39df62499f26e0ef94100b5feb62\ibm_cos_sdk-2.5.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c4/41/523f6a05e6dc3329a5660f6a81254c6cd87e5cfb5b7482bae3391d86ec3a/tabulate-0.8.6.tar.gz
Processing c:\users\usuario\appdata\local\pip\cache\wheels\0e\f5\95\9c4a7d23860dc34fe0a49f2265b5259802d2ad72156fd9a6fe\ibm_cos_sdk_core-2.5.5-py2.py3-none-any.whl
Processing c:\users\usuario\appdata\local\pip\cache\wheels\e9\80\1a\6bdcf3777571ab7a36539a65f0fc95c13d347a7162f32bd5ce\ibm_cos_sdk_s3transfer-2.5.5-py2.py3-none-any.whl
  Using cached https://files.pyt

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [6]:
credentials = {
  "apikey": "BWXQlEvK1ASCLjOQDJSidnc6IV9LbN3eEunn358kmrd9",
  "iam_apikey_description": "Auto-generated for key 69933e1b-ad1b-4b9a-8325-ed725c516183",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/bab206838d6e406f9cdf028c8c396869::serviceid:ServiceId-c317915f-dc41-473c-abf6-0f2b4a56e085",
  "instance_id": "52a70d04-6d3a-4846-b0a5-55f3bb9c28ca",
  "url": "https://us-south.ml.cloud.ibm.com"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [7]:
client = WatsonMachineLearningAPIClient(credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [8]:
metadata = { 
            client.repository.ModelMetaNames.NAME: 'Sistema_de_recomendacion',
            client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
            client.repository.ModelMetaNames.RUNTIME_VERSION: '3.6',
            client.repository.ModelMetaNames.FRAMEWORK_NAME: 'scikit-learn',
            client.repository.ModelMetaNames.AUTHOR_NAME: 'GauchatLuciano',
}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vec = TfidfVectorizer()

pipeline = make_pipeline(tfid_vec, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...100, random_state=None))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [10]:
import pandas as pd

In [11]:
online_model = client.repository.store_model(
                                                model= pipeline,
                                                meta_props = metadata,
                                                training_data = X_train,
                                                training_target = y_train
)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [12]:
models_details = client.repository.list_models()

------------------------------------  ------------------------  ------------------------  -----------------
GUID                                  NAME                      CREATED                   FRAMEWORK
12f7b892-04b6-4433-9ef5-e050759940cc  Sistema_de_recomendacion  2019-11-26T01:50:08.500Z  scikit-learn-0.19
------------------------------------  ------------------------  ------------------------  -----------------


In [13]:
guid = client.repository.get_model_uid(online_model)

**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [14]:
model = client.repository.load(guid)

predictions = model.predict(X_test)


C:\Users\USUARIO\Anaconda3\envs\Proyecto 7\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


**9) Mostrar** el `classification_report` obtenido

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.82      0.90      0.86       200
          1       0.89      0.80      0.84       200

avg / total       0.85      0.85      0.85       400

